# Preparação Dos Dados


In [10]:
import pandas as pd
import numpy as np
import os

# Leitura dos dados originais
# Mostrar todas as colunas
pd.set_option('display.max_columns', None)
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Função para calcular derivadas percentuais
def calcular_derivadas(data):
    data['V1'] = data['Vazao'].pct_change() * 100  # Primeira derivada
    data['V2'] = data['V1'].diff()  # Segunda derivada
    return data

# Função para criar defasagens
def criar_defasagens(data, colunas, entradas):
    defasagens_list = []

    for col in colunas:
        for i in range(entradas, 0, -1):
            defasagens_list.append(data[col].shift(i).rename(f'Entrada_{col}_{i}'))
        defasagens_list.append(data[col])

    for i in range(1, 12):
        defasagens_list.append(data['Vazao'].shift(-i).rename(f'Saida_{i}'))
    defasagens = pd.concat(defasagens_list, axis=1)
    return defasagens


# Função para criar colunas binárias
def criar_colunas_binarias(data):
    # Inicializar as colunas binárias com zeros
    data['Bin1'] = 0
    data['Bin2'] = 0
    data['Bin3'] = 0
    
    # Aplicar as condições para definir os valores das colunas binárias
    data.loc[(data['V1'] > 0.4) & (data['V2'] > 0), ['Bin1', 'Bin2', 'Bin3']] = [1, 0, 0]
    data.loc[(data['V1'] < -0.4) & (data['V2'] < 0), ['Bin1', 'Bin2', 'Bin3']] = [0, 0, 1]
    data.loc[~((data['V1'] > 0.4) & (data['V2'] > 0)) & ~((data['V1'] < -0.4) & (data['V2'] < 0)), ['Bin1', 'Bin2', 'Bin3']] = [0, 1, 0]
    
    return data

# **MODELO 1**

In [ ]:
# Criar datasets com defasagem apenas na coluna 'Vazao'
data_15 = criar_defasagens(data.copy(), ['Vazao'], 15)
data_30 = criar_defasagens(data.copy(), ['Vazao'], 30)
data_45 = criar_defasagens(data.copy(), ['Vazao'], 45)

# Pasta onde os arquivos serão salvos

output_folder = 'data/modelo_1'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_15 = os.path.join(output_folder, 'data_15_dias.xlsx')
file_path_30 = os.path.join(output_folder, 'data_30_dias.xlsx')
file_path_45 = os.path.join(output_folder, 'data_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_15.to_excel(file_path_15, index=False)
data_30.to_excel(file_path_30, index=False)
data_45.to_excel(file_path_45, index=False)

print("DataFrames salvos com sucesso!")

# **MODELO 2**



In [ ]:
# Calcular derivadas
data_com_derivadas = calcular_derivadas(data)

# Criar datasets com defasagem nas colunas 'V2', 'V1' e 'Vazao'
data_derivadas_15 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 15)
data_derivadas_30 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 30)
data_derivadas_45 = criar_defasagens(data_com_derivadas, ['V2', 'V1','Vazao'], 45)

# Pasta onde os arquivos serão salvos, dentro de 'data'
output_folder = 'data/modelo_2'

# Criar a pasta 'modelo_2' dentro de 'data' se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos  para os arquivos Excel
file_path_derivadas_15 = os.path.join(output_folder, 'data_derivadas_15_dias.xlsx')
file_path_derivadas_30 = os.path.join(output_folder, 'data_derivadas_30_dias.xlsx')
file_path_derivadas_45 = os.path.join(output_folder, 'data_derivadas_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_derivadas_15.to_excel(file_path_derivadas_15, index=False)
data_derivadas_30.to_excel(file_path_derivadas_30, index=False)
data_derivadas_45.to_excel(file_path_derivadas_45, index=False)

print("DataFrames salvos com sucesso!")

DataFrames salvos com sucesso!


In [13]:
file_path = 'data/modelo_2/data_derivadas_15_dias.xlsx'
data1 = pd.read_excel(file_path,usecols=['V2','V1','Vazao'])
data1

,V2,V1,Vazao
0,NaN,NaN,2110
1,NaN,-0.995261,2089
2,-0.871661,-1.866922,2050
3,-0.084298,-1.951220,2010
4,1.005946,-0.945274,1991
...,...,...,...
17284,-1.364289,0.389105,5418
17285,-3.028455,-2.639350,5275
17286,-1.531266,-4.170616,5055
17287,-0.517811,-4.688427,4818


# **MODELO 3**



In [11]:
# Adicionar colunas binárias ao DataFrame
data_com_derivadas = calcular_derivadas(data)
data_com_binarias = criar_colunas_binarias(data_com_derivadas).dropna()

data_com_binarias[['Bin1','Bin2','Bin3','V2','V1']].head(10)



,Bin1,Bin2,Bin3,V2,V1
2,0,0,1,-0.871661,-1.866922
3,0,0,1,-0.084298,-1.951220
4,0,1,0,1.005946,-0.945274
5,0,0,1,-0.059247,-1.004520
6,0,1,0,1.004520,0.000000
7,0,1,0,0.000000,0.000000
8,1,0,0,0.507357,0.507357
9,0,1,0,-0.507357,0.000000
10,1,0,0,0.959112,0.959112
11,0,1,0,-0.959112,0.000000


In [12]:
# Criar datasets com defasagem nas colunas binárias e 'Vazao'
data_binarias_15 = criar_defasagens(data_com_binarias, ['Bin1', 'Bin2', 'Bin3', 'Vazao'], 15)
data_binarias_30 = criar_defasagens(data_com_binarias, ['Bin1', 'Bin2', 'Bin3', 'Vazao'], 30)
data_binarias_45 = criar_defasagens(data_com_binarias, ['Bin1', 'Bin2', 'Bin3', 'Vazao'], 45)

# Pasta onde os arquivos serão salvos, dentro de 'data'
output_folder = 'data/modelo_3'

# Criar a pasta 'modelo_3' dentro de 'data' se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_binarias_15 = os.path.join(output_folder, 'data_binarias_15_dias.xlsx')
file_path_binarias_30 = os.path.join(output_folder, 'data_binarias_30_dias.xlsx')
file_path_binarias_45 = os.path.join(output_folder, 'data_binarias_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel
data_binarias_15.to_excel(file_path_binarias_15, index=False)
data_binarias_30.to_excel(file_path_binarias_30, index=False)
data_binarias_45.to_excel(file_path_binarias_45, index=False)

print("DataFrames salvos com sucesso!")


DataFrames salvos com sucesso!


In [43]:
file_path = 'data/modelo_3/data_binarias_15_dias.xlsx'
data3 = pd.read_excel(file_path,usecols=['Bin1','Bin2','Bin3','Vazao'])
data3

,Bin1,Bin2,Bin3,Vazao
0,0,1,0,2110
1,0,1,0,2089
2,0,0,1,2050
3,0,0,1,2010
4,0,1,0,1991
...,...,...,...,...
17284,0,1,0,5418
17285,0,0,1,5275
17286,0,0,1,5055
17287,0,0,1,4818
